In [1]:
!pip install konlpy

In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 26 kB/s             
     |████████████████████████████████| 344 kB 1.8 MB/s            
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.25-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp38-cp38-linux_x86_64.whl size=688949 sha256=2c445a55e76d5bf63ece1eca01ff073d797d23cd3f38939a495c0b9dde9015c1
  Stored in directory: /root/.cache/pip/wheels/b6/93/9d/2237550c409eb3ed725d6302b7897ddd9a037b40cef66dcd9c
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 1.8 MB/s            
     |████████████████████████████████| 769 kB 1.8 MB/s            
     |████████████████████████████████| 764 kB 121.7 MB/s            
     |████████████████████████████████| 895 kB 137.1 MB/s            
     |████████████████████████████████| 3.0 MB 119.4 MB/s            
     |████████████████████████████████| 97 kB 10.6 MB/s  

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |██████████████████████████████  | 827.8 MB 162.3 MB/s eta 0:00:01     |████████████████████████████████| 881.9 MB 1.2 kB/s               0:59:38  
ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device



In [3]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-l4rh247v
  Running command git clone --filter=blob:none -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-l4rh247v
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 5b78dacc94c5ba0f7b2fe7ea8c781e17f34ad2af
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 4.9 MB 1.8 MB/s            
     |████████████████████████████████| 3.3 MB 124.5 MB/s            
     |████████████████████████████████| 701 kB 129.1 MB/s            
     |████████████████████████████████| 3.3 MB 109.3 MB/s            
     |████████████████████████████████| 61 kB 537 kB/s              
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=13135 sha256=a0c7013075dedd5d72bfaae88666b8765896001241dc7e1e0b6a2beec3d6d00b
  Stored in directory: /tmp/pip-ephem-wheel-cache-327q_lce/wheels/bf/5f/74/81bf3a1332130eb6629ecf58876a8746b77021e7d7b0638e91


In [6]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

ModuleNotFoundError: No module named 'torch._C'

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from konlpy.tag import Okt  
import tensorflow as tf
import numpy as np
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt


import urllib.request
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
import matplotlib.pyplot as plt 
%matplotlib inline
from string import punctuation
import warnings
warnings.filterwarnings('ignore')

In [ ]:
emotion = pd.read_csv('emotion_dataset.csv')
emotion.head(1)

In [ ]:
emotion.info()

## 전처리

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
emotion 

In [20]:
emotion['Sentence'] = emotion['Sentence'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
# 한글과 공백을 제외하고 모두 제거
emotion[:5]

,Sentence,Emotion
0,언니 동생으로 부르는게 맞는 일인가요,공포
1,그냥 내 느낌일뿐겠지,공포
2,아직너무초기라서 그런거죠,공포
3,유치원버스 사고 낫다던데,공포
4,근데 원래이런거맞나요,공포


In [21]:
emotion['Sentence'] = emotion['Sentence'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
emotion['Sentence'].replace('', np.nan, inplace=True)
print(emotion.isnull().sum())

Sentence    2
Emotion     0
dtype: int64


In [22]:
emotion[emotion['Sentence'].isnull()]

,Sentence,Emotion
10149,NaN,놀람
31822,NaN,행복


In [23]:
emotion = emotion.dropna(how = 'any')
print('전처리 후 데이터의 개수 :',len(emotion))

전처리 후 데이터의 개수 : 38592


In [24]:
emotion.loc[(emotion['Emotion'] == "공포"), 'Emotion'] = 0  #공포 => 0
emotion.loc[(emotion['Emotion'] == "놀람"), 'Emotion'] = 1  #놀람 => 1
emotion.loc[(emotion['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
emotion.loc[(emotion['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
emotion.loc[(emotion['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
emotion.loc[(emotion['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
emotion.loc[(emotion['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

In [25]:
data_list = []
for q, label in zip(emotion['Sentence'], emotion['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [26]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['언니 동생으로 부르는게 맞는 일인가요', '0']
['기술적으로도 아직도 해체해서 다시 완벽히 돌려놓는게 어려운데 해체를한다고', '1']
['명박이도 조사해라', '2']
['원래 디스크가 좀 있어서허리가 아프  ㅠ ㅠ', '3']
['개고기 레알 밥도둑', '4']
['저에게 요리의 희망을 주신분이기에 사랑할수밖에 없는 멋진분입니다', '5']
['와이프도 그렇고 댓글 다 볼텐데 이휘재 좀 하차 하라고 전해주세요', '6']


## Train data & Test data

In [27]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [28]:
print(len(dataset_train))
print(len(dataset_test))

28944
9648


## KoBERT 입력 데이터로 만들기

In [29]:
# BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [30]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [31]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [32]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [33]:
data_train[0]

(array([   2, 2849, 7848, 7234, 5760, 2643,  517, 7028, 7101,  517, 5712,
        5366, 7227, 5859, 5666, 6827, 6701,    3,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(18, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 3)

In [34]:
#  torch 형식의 dataset 생성
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

## KoBERT 학습모델 만들기

> 다중분류할 클래스 수 만큼 num_classes 변수를 수정



In [35]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [36]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [37]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [38]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [39]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [40]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [41]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [42]:
train_dataloader

## KoBERT 모델 학습

In [43]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/453 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.9119298458099365 train acc 0.203125
epoch 1 batch id 201 loss 1.3554713726043701 train acc 0.29905161691542287
epoch 1 batch id 401 loss 1.2080492973327637 train acc 0.39187188279301743
epoch 1 train acc 0.4036630794701987


  0%|          | 0/151 [00:00<?, ?it/s]

epoch 1 test acc 0.4885140728476821


  0%|          | 0/453 [00:00<?, ?it/s]

...

epoch 4 test acc 0.5336989514348787


  0%|          | 0/453 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.7809841632843018 train acc 0.78125
epoch 5 batch id 201 loss 0.6206881999969482 train acc 0.7806281094527363
epoch 5 batch id 401 loss 0.4609687924385071 train acc 0.7960567331670823
epoch 5 train acc 0.796978476821192


  0%|          | 0/151 [00:00<?, ?it/s]

epoch 5 test acc 0.5335954746136866


## 새로운 문장 테스트
훈련시킨 KoBERT 모델에 새로운 문장을 넣어, 감정 분류를 잘 하는지 테스트

In [44]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [45]:
#### 예측을 해주기 위한 predict 함수

In [100]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits_softmax = F.softmax(i)
            logits_softmax = logits_softmax.detach().cpu().numpy()

            if np.argmax(logits_softmax) == 0:
                test_eval.append("공포")
            elif np.argmax(logits_softmax) == 1:
                test_eval.append("놀람")
            elif np.argmax(logits_softmax) == 2:
                test_eval.append("분노")
            elif np.argmax(logits_softmax) == 3:
                test_eval.append("슬픔")
            elif np.argmax(logits_softmax) == 4:
                test_eval.append("중립")
            elif np.argmax(logits_softmax) == 5:
                test_eval.append("행복")
            elif np.argmax(logits_softmax) == 6:
                test_eval.append("혐오")

        print( test_eval[0] + " 감정을 가진 문장입니다.")

In [124]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        emotion = ["공포","놀람","분노","슬픔","중립","행복","혐오"]
        for i in out:
            logits_softmax = F.softmax(i)
            logits_softmax = logits_softmax.detach().cpu().numpy()
            emotion_score={}
            for i, logit in enumerate(logits_softmax):
                emotion_score[emotion[i]]=logit

            if np.argmax(logits_softmax) == 0:
                test_eval.append("공포")
                y= max(emotion_score.values())*100
            elif np.argmax(logits_softmax) == 1:
                test_eval.append("놀람")
                y= max(emotion_score.values())*100
            elif np.argmax(logits_softmax) == 2:
                test_eval.append("분노")
                y= max(emotion_score.values())*100
            elif np.argmax(logits_softmax) == 3:
                test_eval.append("슬픔")
                y= max(emotion_score.values())*100
            elif np.argmax(logits_softmax) == 4:
                test_eval.append("중립")
                y= max(emotion_score.values())*100
            elif np.argmax(logits_softmax) == 5:
                test_eval.append("행복")
                y = max(emotion_score.values())*100
            elif np.argmax(logits_softmax) == 6:
                test_eval.append("혐오")
                y = max(emotion_score.values())*100
#####################
        print( y , '확률로' , test_eval[0] + " 감정을 가진 문장입니다.")

In [96]:
import torch.nn.functional as F

In [132]:
# torch.save(model.state_dict(), 'bert_multi_emotion.h5')

```
model = TheModelClass(*args, **kwargs)
model.load_state_dict(torch.load(PATH))
model.eval()
```

In [105]:
emotion = ["공포","놀람","분노","슬픔","중립","행복","혐오"]
for i in out:
    logits_softmax = F.softmax(i)
    logits_softmax = logits_softmax.detach().cpu().numpy()
    emotion_score={}
    for i, logit in enumerate(logits_softmax):
        emotion_score[emotion[i]]=logit

### 감정 예측

In [134]:
predict('바보처럼 울고 있는 너의 곁에  상처만 주는 나를 왜 모르고')

95.27019262313843 확률로 슬픔 감정을 가진 문장입니다.


In [135]:
predict('창가에 요란히 내리는 빗물 소리만큼 시린 기억들')

97.29878306388855 확률로 슬픔 감정을 가진 문장입니다.


In [137]:
predict('너의 웃음소리 참 듣기가 좋아')

98.91051650047302 확률로 행복 감정을 가진 문장입니다.


In [138]:
predict('나 그댈 위해 시 한 편을 쓰겠어')

55.526894330978394 확률로 중립 감정을 가진 문장입니다.


---

## 감정 분류 및 데이터 프레임 생성

In [139]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        emotion = ["공포","놀람","분노","슬픔","중립","행복","혐오"]
        for i in out:
            logits_softmax = F.softmax(i)
            logits_softmax = logits_softmax.detach().cpu().numpy()
            emotion_score={}
            for i, logit in enumerate(logits_softmax):
                emotion_score[emotion[i]]=logit

            if np.argmax(logits_softmax) == 0:
                test_eval.append("공포")
                y= max(emotion_score.values())*100
            elif np.argmax(logits_softmax) == 1:
                test_eval.append("놀람")
                y= max(emotion_score.values())*100
            elif np.argmax(logits_softmax) == 2:
                test_eval.append("분노")
                y= max(emotion_score.values())*100
            elif np.argmax(logits_softmax) == 3:
                test_eval.append("슬픔")
                y= max(emotion_score.values())*100
            elif np.argmax(logits_softmax) == 4:
                test_eval.append("중립")
                y= max(emotion_score.values())*100
            elif np.argmax(logits_softmax) == 5:
                test_eval.append("행복")
                y = max(emotion_score.values())*100
            elif np.argmax(logits_softmax) == 6:
                test_eval.append("혐오")
                y = max(emotion_score.values())*100
#####################
        return [y, test_eval[0]]

In [145]:
result = []
for i in range(len(test)):    
    emotion_list = {
    'emotion' : predict(test.Lyric[i])[1],
    'percentage': predict(test.Lyric[i])[0]        
    }

    df = pd.DataFrame.from_dict(emotion_list, orient='index')
    df = df.transpose()
    result.append(df)

In [142]:
playlist = pd.read_csv('pre_total_playlist.csv')

In [147]:
test = playlist[:5]
test

,song_id,song_name,artist,album,Like_Count,Lyric,cover_url,tags,words
0,418253,보고 싶다,김범수,3rd. 보고 싶다,342238,아무리 기다려도 난 못가 바보처럼 울고 있는 너의 곁에 상처만 주는 나를 왜 ...,https://image.bugsm.co.kr/album/images/200/262...,"['국내', '가요', '발라드한', '발라드', '국내 발라드', '가을', '쓸...","['아무리', '기다려도', '난', '못가', '바보처럼', '울고', '있는',..."
1,445413,사랑합니다...,팀,Tim 영민,332810,나빠요 참 그대란 사람 허락도 없이 왜 내 맘 가져요 그대 때문에 난 힘겹게 살고만...,https://image.bugsm.co.kr/album/images/200/307...,"['국내', '발라드', '발라드한', '100시리즈', '가을', '가요', '감...","['나빠요', '참', '그대란', '사람', '허락도', '없이', '왜', '내..."
2,489056,어쩌면...,버즈,Morning Of Buzz,295142,어쩌면 오늘이 마지막이 될지도 몰라 나의 사랑이 떠날지 몰라 불안한 나의 마음 니가...,https://image.bugsm.co.kr/album/images/200/328...,"['국내', '락발라드', '발라드한', '2000년대', '세부장르', '100시...","['어쩌면', '오늘이', '마지막이', '될지도', '몰라', '나의', '사랑이..."
3,460396,소주 한 잔,임창정,Bye,277349,술이 한 잔 생각나는 밤 같이 있는 것 같아요 그 좋았던 시절들 이젠 모두 한숨만...,https://image.bugsm.co.kr/album/images/200/314...,"['국내', '발라드한', '발라드', '국내 발라드', '100시리즈', '가요'...","['술이', '한', '잔', '생각나는', '밤', '같이', '있는', '것',..."
4,482653,With Me,휘성(Realslow),It's Real,268787,네 손짓 하나 보는 게 난 좋은데 네 목소리를 듣는 것도 좋은데 왜 넌 내 ...,https://image.bugsm.co.kr/album/images/200/325...,"['가요', '섹시한', '남성보컬']","['네', '손짓', '하나', '보는', '게', '난', '좋은데', '네', ..."


In [146]:
emotion_df = pd.concat(result).reset_index(drop=True)    
emotion_df

,emotion,percentage
0,슬픔,97.5559
1,슬픔,95.2685
2,슬픔,95.931
3,슬픔,88.2449
4,슬픔,96.9221


---

In [ ]:
result = []
for i in tqdm(range(len(playlist))):    
    emotion_list = {
    'emotion' : predict(playlist.Lyric[i])[1],
    'percentage': predict(playlist.Lyric[i])[0]        
    }

    df = pd.DataFrame.from_dict(emotion_list, orient='index')
    df = df.transpose()
    result.append(df)

 19%|█▉        | 2957/15342 [1:14:46<8:36:47,  2.50s/it]

In [ ]:
emotion_df = pd.concat(result).reset_index(drop=True)    
emotion_df

In [ ]:
emotion_df.to_csv('bert_eclf_playlist.csv',index=False)

In [ ]:
years = pd.read_csv('sort_total_years.csv')